In [1]:
# Requirements:
# !pip install rtdl
# !pip install libzero==0.0.4

In [2]:
from typing import Any, Dict

import numpy as np
import rtdl
import scipy.special
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
import zero
import pandas as pd

In [3]:
device = torch.device('cpu')
# Docs: https://yura52.github.io/zero/0.0.4/reference/api/zero.improve_reproducibility.html
zero.improve_reproducibility(seed=123456)

123456

### Data

In [4]:
from sklearn.datasets import fetch_kddcup99 
data = fetch_kddcup99(subset='train')

from pprint import pprint
print(list(data.target_names))

['labels']


In [5]:
df = pd.read_csv("data/adult.csv")
df["target"] = df.income
df = df.drop("income", axis=1)

In [6]:
strCol = []
for column in df.columns:
    print(column)
    if type(df[column][0]) is str:
        strCol.append(column)


age
workclass
fnlwgt
education
education.num
marital.status
occupation
relationship
race
sex
capital.gain
capital.loss
hours.per.week
native.country
target


In [7]:
# df = pd.DataFrame(data.data.tolist(), columns=data.feature_names)
oldNames = df.columns# data.feature_names

In [8]:
# df["target"] = data.target
# df["target"] = df.income

In [9]:
output = df.target.values
labels = set(output)
print('The different type of output labels are:',labels)
print('='*125)
print('No. of different output labels are:', len(labels))

The different type of output labels are: {'<=50K', '>50K'}
No. of different output labels are: 2


In [10]:
def onhotencode(_df, _col):
    _values = set(_df[_col].values)
    for v in _values:
        _df[v] = _df[_col].apply(lambda x : float(x == v) )
    return _df

In [11]:
for c in df.columns:
    if not c in strCol :
        df = df.drop(c, axis=1)

In [12]:
for col in df.columns:
    if col != "target":
        df = onhotencode(df, col)
        df = df.drop(col, axis=1)

In [13]:
# df = onhotencode(df, col)

In [14]:
df["target"] = df["target"].apply(lambda x: str(x))

In [15]:
df.to_csv("data/adult_processed.csv")

In [16]:
df

,target,Private,Self-emp-inc,Local-gov,State-gov,?,Without-pay,Federal-gov,Never-worked,Self-emp-not-inc,...,China,Germany,India,Dominican-Republic,Ecuador,Poland,Hungary,Holand-Netherlands,Mexico,Peru
0,<=50K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,<=50K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,<=50K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,<=50K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,<=50K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,<=50K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32557,<=50K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32558,>50K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32559,<=50K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
list(df.columns)

['target',
 'Private',
 'Self-emp-inc',
 'Local-gov',
 'State-gov',
 '?',
 'Without-pay',
 'Federal-gov',
 'Never-worked',
 'Self-emp-not-inc',
 'Some-college',
 'Bachelors',
 '11th',
 '9th',
 'Preschool',
 'Assoc-acdm',
 'Masters',
 'Doctorate',
 '7th-8th',
 '5th-6th',
 '1st-4th',
 'HS-grad',
 '10th',
 'Assoc-voc',
 '12th',
 'Prof-school',
 'Separated',
 'Married-spouse-absent',
 'Divorced',
 'Never-married',
 'Married-civ-spouse',
 'Married-AF-spouse',
 'Widowed',
 'Priv-house-serv',
 'Armed-Forces',
 'Machine-op-inspct',
 'Tech-support',
 'Transport-moving',
 'Sales',
 'Farming-fishing',
 'Prof-specialty',
 'Protective-serv',
 'Exec-managerial',
 'Handlers-cleaners',
 'Other-service',
 'Craft-repair',
 'Adm-clerical',
 'Own-child',
 'Wife',
 'Other-relative',
 'Not-in-family',
 'Husband',
 'Unmarried',
 'Asian-Pac-Islander',
 'Other',
 'Black',
 'White',
 'Amer-Indian-Eskimo',
 'Female',
 'Male',
 'Philippines',
 'Scotland',
 'Canada',
 'El-Salvador',
 'Iran',
 'Yugoslavia',
 'Japan